In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
df_test = pd.read_csv('merged_CSVs.csv')
# df_test_1 = df_test.head(5)
# display(df_test_1)

<ipython-input-2-90b701f92e02>:1: DtypeWarning: Columns (3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267

In [3]:
num_rows = df_test.shape[0]
print(f"The number of rows is: {num_rows}")

The number of rows is: 700776


In [4]:
columns_to_print = [
    'total_header_bytes', 'max_header_bytes', 'min_header_bytes',
    'mean_header_bytes', 'mode_header_bytes', 'fwd_total_header_bytes',
    'fwd_mode_header_bytes', 'fwd_init_win_bytes', 'bwd_init_win_bytes',
    'packets_rate', 'packet_IAT_total', 'packets_IAT_median',
    'fwd_packets_IAT_mean', 'bwd_packets_IAT_mode', 'label'
]
row_294096 = df_test[columns_to_print].iloc[294096]  # 294097th row is at index 294096 (since index is zero-based)
print(row_294096)
row_294097 = df_test[columns_to_print].iloc[294097]  # 294097th row is at index 294096 (since index is zero-based)
print(row_294097)

total_header_bytes                       364
max_header_bytes                          32
min_header_bytes                          20
mean_header_bytes                    21.4118
mode_header_bytes                       20.0
fwd_total_header_bytes                   212
fwd_mode_header_bytes                   20.0
fwd_init_win_bytes                      8192
bwd_init_win_bytes                     64000
packets_rate                4.72051102657086
packet_IAT_total          3.6013050079345703
packets_IAT_median                    0.0787
fwd_packets_IAT_mean                  0.4001
bwd_packets_IAT_mode                  0.0795
label                                 Benign
Name: 294096, dtype: object
total_header_bytes                       20
max_header_bytes                         20
min_header_bytes                         20
mean_header_bytes                      20.0
mode_header_bytes                      20.0
fwd_total_header_bytes                   20
fwd_mode_header_bytes            

In [6]:
import pandas as pd
import os
from tqdm import tqdm

input_csv = "merged_CSVs.csv"
output_csv = "llama-2-cpacket-data.csv"
total_rows = 50000  # Desired number of rows for the final dataset
chunk_size = 10000  # Number of rows to write per chunk

# Function to convert a row into the desired text format for LLaMA-2
def row_to_text(row):
    try:
        # User's message based on network flow data
        user_message = (
            f"The network flow has total header bytes of {row['total_header_bytes']} and "
            f"max header bytes of {row['max_header_bytes']} and min header bytes of {row['min_header_bytes']}. "
            f"The mean header bytes are {float(row['mean_header_bytes']):.2f} while the mode is {row['mode_header_bytes']}. "
            f"The forward total header bytes are {row['fwd_total_header_bytes']} and forward mode header bytes are {row['fwd_mode_header_bytes']}. "
            f"The forward initial window bytes are {row['fwd_init_win_bytes']} and backward initial window bytes are {row['bwd_init_win_bytes']}. "
            f"The packet rate is {float(row['packets_rate']):.2f}. The inter-arrival time total is {float(row['packet_IAT_total']):.10f} "
            f"with a median of {float(row['packets_IAT_median']):.10f}. The forward packet IAT mean is {float(row['fwd_packets_IAT_mean']):.10f} "
            f"while the backward packet IAT mode is {float(row['bwd_packets_IAT_mode']):.10f}. "
            f"Is this network data flow associated with any DDoS attacks?"
        )

        # Model's response (LLM's response)
        model_response = f"This data flow can be categorized as {row['label']}."

        # LLaMA-2 format with <s>[INST] tokens
        formatted_text = f"<s>[INST] {user_message} [/INST] {model_response} </s>"
        return formatted_text

    except ValueError as e:
        print(f"Error processing row: {row}, error: {e}")
        return ""

# Load the entire CSV file into a dataframe
df = df_test

# Get the unique labels and calculate the required number of rows per label
unique_labels = df['label'].unique()
rows_per_label = total_rows // len(unique_labels)  # Equal rows for each label

# Sample rows equally for each label
sampled_dfs = []
for label in unique_labels:
    # Filter rows for the current label
    label_df = df[df['label'] == label]
    # If the label has fewer rows than required, sample with replacement
    sampled_label_df = label_df.sample(n=rows_per_label, replace=True, random_state=42)
    sampled_dfs.append(sampled_label_df)

# Combine all sampled rows into a single dataframe
final_df = pd.concat(sampled_dfs)

# Shuffle the final dataframe to mix the rows
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Open the output file in write mode
with open(output_csv, "w") as output_file:
    output_file.write("text\n")  # Write the header

    # Process the final dataframe in chunks and write to the output CSV
    for i in tqdm(range(0, len(final_df), chunk_size)):
        chunk = final_df.iloc[i:i + chunk_size]

        # Convert each row to the LLaMA-2 format and write to file
        for index, row in chunk.iterrows():
            text = row_to_text(row)
            if text:  # Write only if text is non-empty
                output_file.write(f"{text}\n")

print(f"Successfully written {total_rows} rows to {output_csv}")

  0%|          | 0/5 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 8282, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                               

 20%|██        | 1/5 [00:08<00:32,  8.01s/it]

Streaming output truncated to the last 5000 lines.
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 18230, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                              

 40%|████      | 2/5 [00:13<00:20,  6.73s/it]

...

Streaming output truncated to the last 5000 lines.
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 38210, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                                            src_port
dst_ip                                                

 80%|████████  | 4/5 [00:25<00:06,  6.18s/it]

Streaming output truncated to the last 5000 lines.
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 48088, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                              

100%|██████████| 5/5 [00:32<00:00,  6.58s/it]

Error processing row: flow_id                                                              flow_id
timestamp                                                          timestamp
src_ip                                                                src_ip
src_port                                                            src_port
dst_ip                                                                dst_ip
                                                        ...                 
median_fwd_payload_bytes_delta_len        median_fwd_payload_bytes_delta_len
skewness_fwd_payload_bytes_delta_len    skewness_fwd_payload_bytes_delta_len
cov_fwd_payload_bytes_delta_len              cov_fwd_payload_bytes_delta_len
label                                                                  label
activity                                                            activity
Name: 49707, Length: 324, dtype: object, error: could not convert string to float: 'mean_header_bytes'
Error processing row: flow_i